In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
%matplotlib inline

In [7]:
sub = 'sub-12'
ses = 'ses-01'

battery = 'camcan'
# task_source = 'gng'
task = 'stop-nogo'

main_dir = '/home/sshankar/ibc/analysis_pipeline/ibc_main/neurospin_data/info/'
datadir = os.path.join(main_dir, sub, battery, ses)
os.chdir(datadir)
csv_files = sorted(glob.glob(task + '*.txt'))
tsv_files = []
i = 1
# for i in range(len(csv_files)):
tsv_files.append(task + '_' + sub + '_run-0' + str(i+1) + '.tsv')

In [3]:
csv_files

['stop-nogo_sub-12_run-01_21621-952.txt',
 'stop-nogo_sub-12_run-02_21621-105.txt']

In [9]:
tsv_files

['stop-nogo_sub-05_run-02.tsv']

In [8]:
# Read in the rest of the file
file_df = pd.read_csv(os.path.join(datadir, csv_files[i]), sep='\t', skipinitialspace=True)

In [9]:
# Extract the onsets of stimulus, probe and response
trial_onsets = (file_df['relStart'] + 500)

# Find the 4 trials types:
# 1. Go trials ('type' = 0)
# 2. No-go trials ('type' = 1 and 'Stype' = 1)
# 3. Successful stop trials ('type' = 1 and 'Stype' = 2 or 3 and RT1 = 0 and RT2 = 0)
# 3. Unsuccessful stop trials ('type' = 1 and 'Stype' = 2 or 3 and (RT1 != 0 or RT2 != 0))
go_trials = file_df['type'] == 0
unsucgo_trials = file_df.loc[go_trials][file_df['RT'] == 0].index
nogo_trials = (file_df['type'] == 1) & (file_df['Stype'] == 1)
unsucnogo_trials = file_df.loc[nogo_trials][file_df['RT'] != 0].index
sucstop_trials = (file_df['type'] == 1) & ((file_df['Stype'] == 2) | (file_df['Stype'] == 3)) & \
            (file_df['RT2'] == 0)
unsucstop_trials = (file_df['type'] == 1) & ((file_df['Stype'] == 2) | (file_df['Stype'] == 3)) & \
            (file_df['RT2'] != 0)

# Calculate trial durations
durations = np.array(np.ones(len(trial_onsets)), dtype=np.int_)
durations[go_trials] = file_df.loc[go_trials]['RT']
durations[unsucgo_trials] = 2500
durations[nogo_trials] = 2500
durations[unsucnogo_trials] = file_df.loc[unsucnogo_trials]['RT']
durations[sucstop_trials] = 2500
durations[unsucstop_trials] = file_df.loc[unsucstop_trials]['RTsum']

# Assign trial types
trial_types = np.array(np.ones(len(trial_onsets)), dtype=np.object_)
trial_types[go_trials] = 'go'
trial_types[nogo_trials] = 'nogo'
trial_types[sucstop_trials] = 'successful_stop'
trial_types[unsucstop_trials] = 'unsuccessful_stop'

/home/sshankar/env/fastsrm/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.
/home/sshankar/env/fastsrm/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':


In [10]:
# Create a DataFrame to save to events file
df = pd.DataFrame({'onset':trial_onsets/1000, 'duration':durations/1000, 'trial_type':trial_types})
df.to_csv(os.path.join(datadir, tsv_files[0]), sep='\t', float_format='%0.3f', index=False)